<a href="https://colab.research.google.com/github/adigew/Breast-Cancer-Detection/blob/main/Resnet101andML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from tensorflow import keras
import cv2
from tensorflow.keras.preprocessing.image import load_img ,img_to_array
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D,AveragePooling2D,BatchNormalization
from tensorflow.keras.layers import Dense,Activation
from tensorflow.keras.layers import Flatten,Dropout,SpatialDropout2D,AveragePooling2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
path = "/content/gdrive/My Drive/Breast_Cancer/Ultrasound/Dataset_BUSI_with_GT"
dir_list = [os.path.join(path,i) for i in os.listdir(path)]
size_dict = {}
for i,value in enumerate(dir_list):
    size_dict[os.listdir(path)[i]] = len(os.listdir(value))
size_dict 

{'normal': 266, 'malignant': 421, 'benign': 891}

In [ ]:
img_size = 128
img_channel = 3
X_b , Xm_b , y_b = np.zeros((437,img_size,img_size,img_channel)) , np.zeros((437,img_size,img_size,img_channel)) , np.full(437,'benign') 
X_n , Xm_n , y_n = np.zeros((133,img_size,img_size,img_channel)) , np.zeros((133,img_size,img_size,img_channel)) , np.full(133,'normal') 
X_m , Xm_m , y_m = np.zeros((210,img_size,img_size,img_channel)) , np.zeros((210,img_size,img_size,img_channel)) , np.full(210,'malignant')


In [ ]:
img1_path = os.path.join(os.path.join(path,'benign'),os.listdir(os.path.join(path,'benign'))[1])

pil_img = load_img(img1_path,color_mode = 'rgb',target_size=(img_size,img_size))
img = img_to_array(pil_img)
img_shape = img.shape
print(img_shape)

def img_num(filename):
    
    val = 0
    
    for i in range(len(filename)) :
        if filename[i] == '(' :
            while True :
                i += 1
                if filename[i] == ')' :
                    break
                val = (val*10) + int(filename[i])
            break
    
    return val

for tumor_path in dir_list :
    for image in os.listdir(tumor_path) :
        p = os.path.join(tumor_path, image)
        pil_img = load_img(p,color_mode = 'rgb',target_size=(img_size,img_size))         # read image as  grayscale ans resize it
        
        if image[-5] == ')' :                                   #if real image 
            
            if image[0] == 'b' :
                X_b[img_num(image)-1]+= img_to_array(pil_img)  # If image is real add it
            if image[0] == 'n' :                               # to X as benign , normal
                X_n[img_num(image)-1]+= img_to_array(pil_img)  # or malignant.
            if image[0] == 'm' :
                X_m[img_num(image)-1]+= img_to_array(pil_img)
        else :                                                 #else masked image
            
            if image[0] == 'b' :
                Xm_b[img_num(image)-1]+= img_to_array(pil_img)  # Similarly add the target
            if image[0] == 'n' :                               # mask to y.
                Xm_n[img_num(image)-1]+= img_to_array(pil_img)
            if image[0] == 'm' :
                Xm_m[img_num(image)-1]+= img_to_array(pil_img)

(128, 128, 3)


In [ ]:
X = np.concatenate((X_b, X_n, X_m), axis = 0)
Xm = np.concatenate((Xm_b, Xm_n, Xm_m), axis = 0)
y = np.concatenate((y_b, y_n, y_m), axis = 0)

print(X.shape)
print(Xm.shape)
print(y.shape)
X /= 255.0
Xm /= 255.0

print(X.max())
print(Xm.min())

(780, 128, 128, 3)
(780, 128, 128, 3)
(780,)
1.0
0.0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

encoder  = OneHotEncoder()
# y = y.toarray()
y=encoder.fit_transform(y.reshape(y.shape[0],1))

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.15,shuffle=True)
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.1,stratify = y_train.toarray())
Xm_train,Xm_test,ym_train,ym_test = train_test_split(Xm,y,test_size = 0.15,shuffle=True,stratify=y.toarray())
Xm_train,Xm_val,ym_train,ym_val = train_test_split(Xm_train,ym_train,test_size=0.1,stratify = ym_train.toarray())

class_list = encoder.categories_
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(596, 128, 128, 3) (117, 128, 128, 3)
(596, 3) (117, 3)


In [ ]:
base_model = tf.keras.applications.ResNet101(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=img_shape,
    pooling=None
)

X_feat_out = base_model.output
X_feat_flatten = Flatten()(X_feat_out)

X_feat_model = Model(inputs = base_model.input,outputs = X_feat_flatten)
X_feat_train = X_feat_model.predict(X_train)
X_feat_val = X_feat_model.predict(X_val)
X_feat_test = X_feat_model.predict(X_test)

Xm_feat_train = X_feat_model.predict(Xm_train)
Xm_feat_val = X_feat_model.predict(Xm_val)
Xm_feat_test = X_feat_model.predict(Xm_test)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.layers import LSTM,Bidirectional,Reshape
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,roc_auc_score,cohen_kappa_score

In [ ]:
df = pd.DataFrame(columns = ['classifier',"train_accuracy",'val_accuracy',"test_accuracy","f1_measure","kappa_score","recall","Precision"])


In [ ]:
def eval(classifier_name,y_train,y_train_pred,y_val,y_val_pred,y_true,y_pred):
    y_train = np.argmax(y_train,axis=1)
#     y_train_pred = np.argmax(y_train_pred,axis=1)
    y_val = np.argmax(y_val,axis=1)
#     y_val_pred = np.argmax(y_val_pred,axis=1)
    y_true = np.argmax(y_true,axis=1)
#     y_pred = np.argmax(y_pred,axis=1)
    
    train_accuracy = round(accuracy_score(y_train,y_train_pred),4)
    val_accuracy = round(accuracy_score(y_val,y_val_pred),4)
    test_accuracy = round(accuracy_score(y_true,y_pred),4)
    f1_measure = round(f1_score(y_true,y_pred,average='weighted'),4)
    kappa_score = round(cohen_kappa_score(y_true,y_pred),4)
    recall = round(recall_score(y_true,y_pred,average='weighted'),4)
    precision = round(precision_score(y_true,y_pred,average='weighted'),4)
    
    score = {"classifier":classifier_name,"train_accuracy":train_accuracy , "val_accuracy":val_accuracy,"test_accuracy":test_accuracy,"f1_measure":f1_measure,"kappa_score":kappa_score,"recall":recall,"precision":precision}
#     df = df.append(score,ignore_index=True)
    df.loc[len(df.index)] = score.values()
    for e,a in score.items():
        print(e,a)
    print("--"*20)
    

In [ ]:
names = ['SVM',
    'Random Forest',
    'AdaBoost',
    'KNN',
    'XGBoost',
    'Bagging',
    'ANN'
        ]

classifier = [
    SVC(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    KNeighborsClassifier(),
    XGBClassifier(),
    BaggingClassifier(),
    MLPClassifier(max_iter = 400),
]

cls_list = zip(names,classifier)
clsm_list = zip(names,classifier)

In [ ]:
def classifier_eval(classifier,classifier_name,X_train,y_train,X_val,y_val,X_test,y_test):
    
    classifier.fit(X_train,np.argmax(y_train,axis=1))
    
    y_train_pred = classifier.predict(X_train)
    y_val_pred = classifier.predict(X_val)
    y_test_pred = classifier.predict(X_test)
    
    eval(classifier_name,y_train,y_train_pred,y_val,y_val_pred,y_test,y_test_pred)

In [ ]:
for n,c in cls_list:
    classifier_eval(c,n,X_feat_train,y_train.toarray(),X_feat_val,y_val.toarray(),X_feat_test,y_test.toarray())

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


classifier SVM
train_accuracy 0.5638
val_accuracy 0.5672
test_accuracy 0.5385
f1_measure 0.3769
kappa_score 0.0
recall 0.5385
precision 0.2899
----------------------------------------
classifier Random Forest
train_accuracy 1.0
val_accuracy 0.7164
test_accuracy 0.735
f1_measure 0.7153
kappa_score 0.5032
recall 0.735
precision 0.7801
----------------------------------------
classifier AdaBoost
train_accuracy 0.7903
val_accuracy 0.6716
test_accuracy 0.6752
f1_measure 0.6697
kappa_score 0.4271
recall 0.6752
precision 0.6785
----------------------------------------
classifier KNN
train_accuracy 0.7903
val_accuracy 0.6418
test_accuracy 0.6325
f1_measure 0.6275
kappa_score 0.3669
recall 0.6325
precision 0.628
----------------------------------------
classifier XGBoost
train_accuracy 1.0
val_accuracy 0.7761
test_accuracy 0.7009
f1_measure 0.6905
kappa_score 0.4623
recall 0.7009
precision 0.7082
----------------------------------------
classifier Bagging
train_accuracy 0.9933
val_accuracy 0.70

In [ ]:
for l in base_model.layers:
    l.trainable = False

In [ ]:
lstm_model = Sequential()
lstm_model.add(base_model)
lstm_model.add(Reshape((base_model.output.shape[1]*base_model.output.shape[2],base_model.output.shape[3])))
lstm_model.add(LSTM(128, dropout=0.5,recurrent_dropout=0.5))
lstm_model.add(Dense(3,activation='softmax'))

lstm_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [ ]:
bidir_model = Sequential()
bidir_model.add(base_model)
bidir_model.add(Reshape((base_model.output.shape[1]*base_model.output.shape[2],base_model.output.shape[3])))
bidir_model.add(Bidirectional(LSTM(128, dropout=0.5,recurrent_dropout=0.5)))
bidir_model.add(Dense(3,activation='softmax'))

bidir_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', patience=4,restore_best_weights=True, verbose=1)

In [ ]:
history = lstm_model.fit(X_train,y_train.toarray(),epochs = 20,validation_data = (X_val,y_val.toarray()),callbacks = [es])
lstm_train_predict = np.argmax(lstm_model.predict(X_train),axis=1)
lstm_val_predict = np.argmax(lstm_model.predict(X_val),axis=1)
lstm_test_predict = np.argmax(lstm_model.predict(X_test),axis=1)
eval("LSTM",y_train,lstm_train_predict,y_val,lstm_val_predict,y_test,lstm_test_predict)

Epoch 1/20
19/19 [==============================] - 80s 4s/step - loss: 1.0283 - acc: 0.5352 - val_loss: 0.9560 - val_acc: 0.5672
Epoch 2/20
19/19 [==============================] - 68s 4s/step - loss: 0.9834 - acc: 0.5638 - val_loss: 0.9468 - val_acc: 0.6269
Epoch 3/20
19/19 [==============================] - 72s 4s/step - loss: 0.9645 - acc: 0.5621 - val_loss: 0.9267 - val_acc: 0.6418
Epoch 4/20
19/19 [==============================] - 68s 4s/step - loss: 0.9329 - acc: 0.5872 - val_loss: 0.9332 - val_acc: 0.5672
Epoch 5/20
19/19 [==============================] - 68s 4s/step - loss: 0.9512 - acc: 0.5789 - val_loss: 1.0005 - val_acc: 0.5672
Epoch 6/20
19/19 [==============================] - 68s 4s/step - loss: 0.9374 - acc: 0.5654 - val_loss: 0.9535 - val_acc: 0.5672
Epoch 7/20
19/19 [==============================] - 68s 4s/step - loss: 0.9330 - acc: 0.5822 - val_loss: 0.8847 - val_acc: 0.6567
Epoch 8/20
19/19 [==============================] - 68s 4s/step - loss: 0.8987 - acc: 0.59

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
history = bidir_model.fit(X_train,y_train.toarray(),epochs = 20,validation_data = (X_val,y_val.toarray()),callbacks = [es])
bidir_train_predict = np.argmax(bidir_model.predict(X_train),axis=1)
bidir_val_predict = np.argmax(bidir_model.predict(X_val),axis=1)
bidir_test_predict = np.argmax(bidir_model.predict(X_test),axis=1)
eval("Bi-dir",y_train,bidir_train_predict,y_val,bidir_val_predict,y_test,bidir_test_predict)

Epoch 1/20
19/19 [==============================] - 85s 4s/step - loss: 1.0041 - acc: 0.5403 - val_loss: 0.9485 - val_acc: 0.5672
Epoch 2/20
19/19 [==============================] - 71s 4s/step - loss: 0.9563 - acc: 0.5671 - val_loss: 0.9241 - val_acc: 0.6716
Epoch 3/20
19/19 [==============================] - 69s 4s/step - loss: 0.9318 - acc: 0.5889 - val_loss: 0.8897 - val_acc: 0.6716
Epoch 4/20
19/19 [==============================] - 71s 4s/step - loss: 0.9055 - acc: 0.6057 - val_loss: 0.8984 - val_acc: 0.6716
Epoch 5/20
19/19 [==============================] - 69s 4s/step - loss: 0.9279 - acc: 0.5856 - val_loss: 0.8754 - val_acc: 0.6716
Epoch 6/20
19/19 [==============================] - 73s 4s/step - loss: 0.8917 - acc: 0.6107 - val_loss: 0.9156 - val_acc: 0.5821
Epoch 7/20
19/19 [==============================] - 69s 4s/step - loss: 0.9108 - acc: 0.5990 - val_loss: 0.8937 - val_acc: 0.6119
Epoch 8/20
19/19 [==============================] - 76s 4s/step - loss: 0.8901 - acc: 0.59

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
df
df.to_excel("RESNET101.xlsx",index=False)

In [ ]:
df = pd.DataFrame(columns = ['classifier',"train_accuracy",'val_accuracy',"test_accuracy","f1_measure","kappa_score","recall","Precision"])
for n,c in clsm_list:
    classifier_eval(c,n,Xm_feat_train,ym_train.toarray(),Xm_feat_val,ym_val.toarray(),Xm_feat_test,ym_test.toarray())

classifier SVM
train_accuracy 0.854
val_accuracy 0.7761
test_accuracy 0.8376
f1_measure 0.822
kappa_score 0.7056
recall 0.8376
precision 0.8635
----------------------------------------
classifier Random Forest
train_accuracy 1.0
val_accuracy 0.8955
test_accuracy 0.8803
f1_measure 0.871
kappa_score 0.7831
recall 0.8803
precision 0.892
----------------------------------------
classifier AdaBoost
train_accuracy 0.8154
val_accuracy 0.7761
test_accuracy 0.7949
f1_measure 0.8007
kappa_score 0.6626
recall 0.7949
precision 0.8196
----------------------------------------
classifier KNN
train_accuracy 0.9161
val_accuracy 0.8806
test_accuracy 0.8376
f1_measure 0.8226
kappa_score 0.7047
recall 0.8376
precision 0.8464
----------------------------------------
classifier XGBoost
train_accuracy 1.0
val_accuracy 0.8507
test_accuracy 0.906
f1_measure 0.901
kappa_score 0.8319
recall 0.906
precision 0.9127
----------------------------------------
classifier Bagging
train_accuracy 0.995
val_accuracy 0.8209

In [ ]:
history = lstm_model.fit(Xm_train,ym_train.toarray(),epochs = 20,validation_data = (Xm_val,ym_val.toarray()),callbacks = [es])
lstm_train_predict = np.argmax(lstm_model.predict(Xm_train),axis=1)
lstm_val_predict = np.argmax(lstm_model.predict(Xm_val),axis=1)
lstm_test_predict = np.argmax(lstm_model.predict(Xm_test),axis=1)
eval("LSTM",ym_train,lstm_train_predict,ym_val,lstm_val_predict,ym_test,lstm_test_predict)

history = bidir_model.fit(Xm_train,ym_train.toarray(),epochs = 20,validation_data = (Xm_val,ym_val.toarray()),callbacks = [es])
bidir_train_predict = np.argmax(bidir_model.predict(Xm_train),axis=1)
bidir_val_predict = np.argmax(bidir_model.predict(Xm_val),axis=1)
bidir_test_predict = np.argmax(bidir_model.predict(Xm_test),axis=1)
eval("Bi-LSTM",ym_train,bidir_train_predict,ym_val,bidir_val_predict,ym_test,bidir_test_predict)

Epoch 1/20
19/19 [==============================] - 68s 4s/step - loss: 1.0835 - acc: 0.4732 - val_loss: 0.8180 - val_acc: 0.5672
Epoch 2/20
19/19 [==============================] - 67s 3s/step - loss: 0.7325 - acc: 0.6795 - val_loss: 0.5338 - val_acc: 0.7313
Epoch 3/20
19/19 [==============================] - 66s 3s/step - loss: 0.5474 - acc: 0.7450 - val_loss: 0.4427 - val_acc: 0.7761
Epoch 4/20
19/19 [==============================] - 67s 4s/step - loss: 0.4382 - acc: 0.8121 - val_loss: 0.4458 - val_acc: 0.8060
Epoch 5/20
19/19 [==============================] - 65s 3s/step - loss: 0.3960 - acc: 0.8490 - val_loss: 0.3529 - val_acc: 0.8657
Epoch 6/20
19/19 [==============================] - 68s 4s/step - loss: 0.3408 - acc: 0.8826 - val_loss: 0.5042 - val_acc: 0.8358
Epoch 7/20
19/19 [==============================] - 66s 3s/step - loss: 0.3130 - acc: 0.8792 - val_loss: 0.3570 - val_acc: 0.8657
Epoch 8/20
19/19 [==============================] - 68s 4s/step - loss: 0.2781 - acc: 0.89

In [ ]:
print(df)
df.to_excel("RESNET101_m.xlsx",index=False)

      classifier  train_accuracy  val_accuracy  test_accuracy  f1_measure  \
0            SVM          0.8540        0.7761         0.8376      0.8220   
1  Random Forest          1.0000        0.8955         0.8803      0.8710   
2       AdaBoost          0.8154        0.7761         0.7949      0.8007   
3            KNN          0.9161        0.8806         0.8376      0.8226   
4        XGBoost          1.0000        0.8507         0.9060      0.9010   
5        Bagging          0.9950        0.8209         0.8803      0.8710   
6            ANN          1.0000        0.8657         0.8974      0.8942   
7           LSTM          0.8758        0.8657         0.8547      0.8389   
8        Bi-LSTM          0.9211        0.8657         0.8889      0.8812   

   kappa_score  recall  Precision  
0       0.7056  0.8376     0.8635  
1       0.7831  0.8803     0.8920  
2       0.6626  0.7949     0.8196  
3       0.7047  0.8376     0.8464  
4       0.8319  0.9060     0.9127  
5       0.783